# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Z-Stage" data-toc-modified-id="Z-Stage-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Z-Stage</a></div><div class="lev1 toc-item"><a href="#Pump" data-toc-modified-id="Pump-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pump</a></div><div class="lev1 toc-item"><a href="#PMT" data-toc-modified-id="PMT-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>PMT</a></div><div class="lev1 toc-item"><a href="#MAX11210-ADC" data-toc-modified-id="MAX11210-ADC-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MAX11210 ADC</a></div>

In [1]:
import asyncio
asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
import logging; logging.basicConfig(level=logging.DEBUG)
import time
import mr_box_peripheral_board as mrbox

proxy = mrbox.SerialProxy(port="COM5")


INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
DEBUG:asyncio:Using proactor: IocpProactor
INFO:listen:listening
DEBUG:asyncio:Using proactor: IocpProactor
DEBUG:read_packets:start listening for packets
DEBUG:read_packets:waiting for packet
INFO:_async_serial_keepalive:Connected to COM5


# Z-Stage

In [2]:
import asyncio
asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
from mr_box_peripheral_board.proxy import SerialProxy
from mr_box_peripheral_board.zstage_ui import ZStageUI
from IPython.display import display

w = ZStageUI(proxy)
display(w.widget)

DEBUG:_send_command:Using timeout 5
DEBUG:read_packets:read: `b'|||\x00\x00d\x00\x1e"\x00\x95\x03\x00\x00\x00\x00\x9d\x03\xcd\xcc\xb8A\xa0\x03\x84\x07\xa8\x03\x01\xb0\x03\xff\xff\xff\x07\xb8\x03'`
DEBUG:read_packets:read: `b'\x00\xa7q'`
DEBUG:read_packets:packet received: DATA
DEBUG:read_packets:parsed packet: `[ 34   0 149   3   0   0   0   0 157   3 205 204 184  65 160   3 132   7
 168   3   1 176   3 255 255 255   7 184   3   0]`
DEBUG:read_packets:waiting for packet
DEBUG:_send_command:Using timeout 5
DEBUG:read_packets:read: `b'|||\x00\x00d\x00'`
DEBUG:read_packets:read: `b'\x00\x00\x00'`
DEBUG:read_packets:packet received: DATA
DEBUG:read_packets:parsed packet: `[]`
DEBUG:read_packets:waiting for packet
DEBUG:_send_command:Using timeout 5
DEBUG:read_packets:read: `b'|||\x00\x00d\x00\x00\x00\x00'`
DEBUG:read_packets:packet received: DATA
DEBUG:read_packets:parsed packet: `[]`
DEBUG:read_packets:waiting for packet
DEBUG:_send_command:Using timeout 5
DEBUG:read_packets:read: `b'|||\

# Pump

In [3]:
from mr_box_peripheral_board import pump_ui
reload(pump_ui)

pump_ui_ = pump_ui.PumpUI(proxy)
pump_ui_.widget


INFO:mr_box_peripheral_board.pump_ui:Pump action completed.


# PMT

In [4]:
import si_prefix as si

from mr_box_peripheral_board import pmt_ui
reload(pmt_ui)

pmt_ui_ = pmt_ui.PmtUI(proxy)
pmt_ui_.widget

# MAX11210 ADC

In [2]:
from mr_box_peripheral_board import max11210_adc_ui
reload(max11210_adc_ui)

# max11210_adc_ui_ = max11210_adc_ui.Max11210AdcUI(proxy)
# max11210_adc_ui_.widget

import ipywidgets as ipw

In [3]:
INPUT_RANGE_UNIPOLAR = 1
INPUT_RANGE_BIPOLAR = 2
CLOCK_SOURCE_EXTERNAL = 1
CLOCK_SOURCE_INTERNAL = 2
FORMAT_OFFSET = 1
FORMAT_TWOS_COMPLEMENT = 2
CONVERSION_MODE_SINGLE = 1
CONVERSION_MODE_CONTINUOUS = 2


def MAX11210_begin(proxy):
    proxy.MAX11210_setDefault();
    proxy.MAX11210_setLineFreq(60);  # 60 Hz
    proxy.MAX11210_setInputRange(INPUT_RANGE_UNIPOLAR);
    proxy.MAX11210_setClockSource(CLOCK_SOURCE_INTERNAL);
    proxy.MAX11210_setEnableRefBuf(True);
    proxy.MAX11210_setEnableSigBuf(True);
    proxy.MAX11210_setFormat(FORMAT_OFFSET);
    proxy.MAX11210_setConvMode(CONVERSION_MODE_SINGLE);
    proxy.MAX11210_selfCal();
    proxy.MAX11210_sysOffsetCal();
    proxy.MAX11210_sysGainCal();

In [7]:
from collections import OrderedDict
import pandas as pd

MAX11210_begin(proxy)
calibration_settings = \
    pd.Series(OrderedDict([('SelfCalGain', proxy.MAX11210_getSelfCalGain()),
                           ('SelfCalOffset', proxy.MAX11210_getSelfCalOffset()),
                           ('SysGainCal', proxy.MAX11210_getSysGainCal()),
                           ('SysOffsetCal', proxy.MAX11210_getSysOffsetCal())]))
print '# Calibration settings #\n'
print calibration_settings

# Calibration settings #

SelfCalGain      16777215
SelfCalOffset    16777215
SysGainCal       16777215
SysOffsetCal     16777215
dtype: int64


In [11]:
print '# Register statuses #\n'
print '----- STAT1 -----\n' + max11210_adc_ui.format_STAT1(proxy.MAX11210_getSTAT1()) + '\n'
print '----- CTRL1 -----\n' + max11210_adc_ui.format_CTRL1(proxy.MAX11210_getCTRL1()) + '\n'
# print '----- CTRL2 -----\n' + str(format(proxy.MAX11210_getCTRL2(),'b')) + '\n'
print '----- CTRL3 -----\n' + max11210_adc_ui.format_CTRL3(proxy.MAX11210_getCTRL3())

# Register statuses #

----- STAT1 -----
Gain Over Range: True
Sampling Rate: 120 Samples/sec
Input Signal Over Max: True
Input Signal Under Min: True
Modulator Status: Busy
ADC Status: Ready


----- CTRL1 -----
Line frequency: 50Hz
Input Range: Unipolar (Positive Only)
Clock: External
Refference Buffer: On
Signal Buffer: On
Format: Offset
Cycle: Single


----- CTRL3 -----
Digital Gain: X2
System Gain: Disabled
System Offset: Disabled
Self-calibration Gain: Disabled
Self-calibration Offset: Disabled



In [6]:
%matplotlib inline
import datetime as dt

import matplotlib as mpl
import matplotlib.ticker
import si_prefix as si
from IPython.display import display


formatter = mpl.ticker.FuncFormatter(lambda x, *args: si.si_format(x, 3))

Vref = 3.0 #Reference Voltage 3.0 V

def _pmt_read(*args):
    
#     proxy.MAX11210_setConvMode(CONVERSION_MODE_SINGLE)
    proxy.MAX11210_setGain(adc_gain.value)
    raw_values = max11210_adc_ui.MAX11210_read(proxy, pmt_rate.value, pmt_duration.value)
    print 'Sampling Rate: ' + str(pmt_rate.value) + ' Hz'
    print 'Digital Gain: ' + str(adc_gain.value)
    voltage = (raw_values / (2 ** 24 - 1))*(Vref/adc_gain.value)
    current = voltage / 30e3  # 30 kOhm
    
    current.to_clipboard()
    
    axis = current.plot()
    axis.yaxis.set_major_formatter(formatter)

    
pmt_rate = ipw.Dropdown(description='Sample rate (Hz)',
                        options=(1, 2, 5, 10, 15, 30, 60, 120), value=1)
adc_gain = ipw.Dropdown(description='Digital Gain',
                        options=(OrderedDict([('X1',1),('X2',2),('X4',4),('X8',8),('X16',16)])), value=1)
pmt_duration = ipw.FloatSlider(description='PMT duration (s)', min=1,
                               max=1000, value=10)

pmt_read = ipw.Button(description='Read PMT')
pmt_read.on_click(_pmt_read)

ipw.VBox([pmt_rate,adc_gain, pmt_duration, pmt_read])

IOError: Did not receive response.